<a href="https://colab.research.google.com/github/Gci04/AML-DS-2021/blob/main/notebooks/AML_Lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generative Adversarial Networks
```
- Advanced Machine Learning, Innopolis University 
- Professor: Muhammad Fahim 
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>


```
Lab Plan
    1. Homework 1 Feedback
    2. Recap Auto-encoder
    3. Vanila GAN
```

<hr>

## 1. Autoencoders (Recap)

* Types of autoencoders
* Applications of autoencoders
* Autoencoders training procedure


![caption](https://miro.medium.com/max/2400/1*Q5dogodt3wzKKktE0v3dMQ@2x.png)

### 1.2 Simple autoencoder

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class autoencoder(nn.Module):
    def __init__(self, input_size, latent_dim):
      super(autoencoder, self).__init__()
      # Step 1 : Define the encoder 
      # Step 2 : Define the decoder
      # Step 3 : Initialize the weights (optional)
      self.encoder = nn.Sequential(
          nn.Linear(input_size, input_size//2),
          nn.ReLU(True),
          nn.Linear(input_size//2, input_size//3),
          nn.Linear(input_size//3, input_size//4),
          nn.Tanh(),
          nn.Linear(input_size//4, latent_dim)
      )
      self.decoder = nn.Sequential(
          nn.Linear(latent_dim, input_size//4),
          nn.ReLU(True),
          nn.Linear(input_size//4, input_size//3),
          nn.Linear(input_size//3, input_size//2),
          nn.Tanh(),
          nn.Linear(input_size//2, input_size)
      )
      self.encoder.apply(self.__init_weights)
      self.decoder.apply(self.__init_weights)
        
    def forward(self, x):
      # Step 1: Pass the input through encoder to get latent representation
      # Step 2: Take latent representation and pass through decoder
      x = self.encoder(x)
      x = self.decoder(x)
      return x
        
        
    
    def encode(self,input):
      #Step 1: Pass the input through the encoder to get latent representation
      return self.encoder(input)
    
    def __init_weights(self,m):
      #Init the weights (optional)
      if type(m) == nn.Linear:
          torch.nn.init.xavier_uniform_(m.weight)
          m.bias.data.fill_(0.01)

### 1.3 Train autoencoder 

In [2]:
# TODO:
# Step 1: Set training parameters (batch size, learning rate, optimizer, number of epochs, loss function)
# Step 2: Create dataset (Randomly generated)
# Step 3: Create data loader 
# Step 4: Define the training loop

batchSize = 100
learning_rate = 0.01
num_epochs = 20
sample = torch.randn((batchSize,1,64))
AE = autoencoder(64,5).to(device)
print(AE)
# print(summary(AE,input_size=(1, 64)))

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(AE.parameters(),lr=learning_rate)

#Create a random dataset
data_loader = DataLoader(TensorDataset(torch.randn((1000,1,64))),batch_size=32,shuffle=True)

for epoch in range(num_epochs):
  epoch_loss = 0.0
  for X in data_loader:
    X = X[0].to(device)

    optimizer.zero_grad()
    # forward
    output = AE(X)
    loss = criterion(output, X)

    # backward
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

  # log
  print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=32, out_features=21, bias=True)
    (3): Linear(in_features=21, out_features=16, bias=True)
    (4): Tanh()
    (5): Linear(in_features=16, out_features=5, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=16, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=16, out_features=21, bias=True)
    (3): Linear(in_features=21, out_features=32, bias=True)
    (4): Tanh()
    (5): Linear(in_features=32, out_features=64, bias=True)
  )
)
epoch [1/20], loss:0.9138
epoch [2/20], loss:1.0399
epoch [3/20], loss:0.9392
epoch [4/20], loss:0.9187
epoch [5/20], loss:0.8835
epoch [6/20], loss:0.8861
epoch [7/20], loss:0.8625
epoch [8/20], loss:0.9260
epoch [9/20], loss:0.9316
epoch [10/20], loss:0.8976
epoch [11/20], loss:0.8641
epoch [12/20], loss:0.9122
epoch [13/20], loss:0.8430
epoch [14/20], loss

## 2. Vannila Generative adversarial network (GAN)

![caption](https://www.researchgate.net/profile/Zhaoqing-Pan/publication/331756737/figure/fig1/AS:736526694621184@1552613056409/The-architecture-of-generative-adversarial-networks.png)

### 2.1 Dataset 

For this lesson we will use SVHN dataset which readily available in `torchvision` and we will do minimal transformation operations 

In [3]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np

import torch
from torchvision import datasets
from torchvision import transforms


def normalize(data_tensor):
    '''re-scale image values to [-1, 1]'''
    return (data_tensor / 255.) * 2. - 1. 

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: normalize(x))])

# SVHN training datasets
svhn_train = datasets.SVHN(root='data/', split='train', download=True, transform=transform)

batch_size = 128
num_workers = 0

# build DataLoaders for SVHN dataset
train_loader = torch.utils.data.DataLoader(dataset=svhn_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=num_workers)

## 2.2 Generator & Discriminator Definition

There are a couple of ways to increase the input of the generator (*z*) to the desired output size.
1. Number of neurones
2. Transposed Convolutions `torch.nn.ConvTranspose2d` [More info](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
#ngf : Number of generator filters
#ndf : Number of discriminator filters
nz = 32
class Discriminator(nn.Module):

    def __init__(self, ndf=3, conv_dim=32):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, 1, 4, 1, 0, bias=False),
            nn.BatchNorm2d(1),
            nn.Flatten(),
            nn.Linear(5*5,1),
            nn.Sigmoid()
          )
        

    def forward(self, x):
        # Step 1: pass the input (real or fake samples) through all hidden layers
        return self.model(x)

class Generator(nn.Module):
    
    def __init__(self, z_size, ngf, conv_dim=32):
        super(Generator, self).__init__()
        # Step 1: Define the generator network architecture
        # NOTE: the input is the random noise size and output is conv_dim i.e (3,32,32)
        self.conv_dim = conv_dim 
        self.input_layer = nn.Linear(in_features=z_size, out_features=2048, bias=True)
        self.model = nn.Sequential(
            nn.ConvTranspose2d(in_channels = 128, out_channels=ngf * 2, kernel_size=4,stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_features= ngf * 2),
            nn.Tanh(),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(num_features=ngf),
            nn.Tanh(),
            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
        

    def forward(self, x):
      # Step 1: pass the input which is random noise to generate the face samples
      x = self.input_layer(x)
      x = x.view(-1, self.conv_dim*4, 4, 4) # (batch_size, depth, 4, 4)
      return self.model(x)

## 2.3 Set hyperparams and training parameters

In [6]:
# define hyperparams
conv_dim = 32
z_size = 100
num_epochs = 10

# define discriminator and generator
D = Discriminator(conv_dim).to(device)
G = Generator(z_size=z_size, ngf=3,conv_dim=conv_dim).to(device)

#print the models summary 
print(D)
print()
print(G)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(64, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (6): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=25, out_features=1, bias=True)
    (9): Sigmoid()
  )
)

Generator(
  (input_layer): Linear(in_features=100, out_features=2048, bias=True)
  (model): Sequential(
    (0): ConvTranspose2d(128, 6, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): ConvTranspose2d(6, 3

## 2.4 Define the loss function for D(x) and G(x)

In [7]:
import torch.optim as optim

def real_loss(D_out, smooth=False):
    batch_size = D_out.size(0)
    # label smoothing
    if smooth:
        # smooth, real labels
        labels = torch.FloatTensor(batch_size).uniform_(0.9, 1).to(device)
    else:
        labels = torch.ones(batch_size).to(device) # real labels = 1     
    
    # binary cross entropy with logits loss
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

def fake_loss(D_out):
    batch_size = D_out.size(0)
    labels = torch.FloatTensor(batch_size).uniform_(0, 0.1).to(device) # fake labels approx 0
    labels = labels.to(device)
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

# params
learning_rate = 0.0003
beta1=0.5
beta2=0.999 # default value

# Create optimizers for the discriminator and generator
d_optimizer = optim.Adam(D.parameters(), learning_rate)
g_optimizer = optim.SGD(G.parameters(), learning_rate)

## 2.5 GAN training Loop

In [ ]:

# keep track of loss and generated, "fake" samples
losses = []

print_every = 300

# Get some fixed data for sampling. These are images that are held
# constant throughout training, and allow us to inspect the model's performance
sample_size=16
fixed_z = torch.FloatTensor(sample_size,z_size).uniform_(-1, 1).to(device)


# train the network
for epoch in range(num_epochs):
  for batch_i, (real_images, _) in enumerate(train_loader):       
    batch_size = real_images.size(0)
    
    
    # TRAIN THE DISCRIMINATOR
    # Step 1: Zero gradients (zero_grad)
    # Step 2: Train with real images
    # Step 3: Compute the discriminator losses on real images 

    d_optimizer.zero_grad()
    real_images = real_images.to(device)
    D_real = D(real_images)
    d_real_loss = real_loss(D_real)
    
    # Step 4: Train with fake images
    # Step 5: Generate fake images and move x to GPU, if available
    # Step 6: Compute the discriminator losses on fake images 
    # Step 7: add up loss and perform backprop

    z = torch.FloatTensor(batch_size,z_size).uniform_(-1, 1).to(device)
    
    fake_images = G(z)
    
    # Compute the discriminator losses on fake images            
    D_fake = D(fake_images)
    d_fake_loss = fake_loss(D_fake)
    
    # add up loss and perform backprop
    d_loss = d_real_loss + d_fake_loss
    d_loss.backward()
    d_optimizer.step()
    
    # fake_images = None     
    
    # d_loss = d_real_loss + d_fake_loss
    # d_loss.backward()
    # d_optimizer.step()
    
    
    #TRAIN THE GENERATOR (Train with fake images and flipped labels)
    g_optimizer.zero_grad()
    
    # Step 1: Zero gradients  
    # Step 2: Generate fake images from random noise (z)
    # Step 3: Compute the discriminator losses on fake images using flipped labels!
    # Step 4: Perform backprop and take optimizer step
    
    # Generate fake images
    z = torch.FloatTensor(batch_size,z_size).uniform_(-1, 1).to(device)

    
    # Compute the discriminator losses on fake images 
    # using flipped labels!
    
    # perform backprop
  

Keep in mind:

1. Always use a learning rate for discriminator higher than the generator.

2. Keep training even if you see that the losses are going up.

3. There are many variations with different loss functions which are worth exploring.

4. If you get mode collapse, lower the learning rates.

5. Adding noise to the training data helps make the model more stable.

6. Label Smoothing: instead of making the labels as 1 make it 0.9 


## References
1. [Understanding Variational Autoencoders (VAEs)](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73)

2. [Intuitively Understanding Variational Autoencoders](https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf)

3. [Tutorial - What is a variational autoencoder?](https://jaan.io/what-is-variational-autoencoder-vae-tutorial/)

4. [The variational auto-encoder](https://ermongroup.github.io/cs228-notes/extras/vae/)

5. [An Introduction to Variational Autoencoders](https://arxiv.org/abs/1906.02691)

6. [Basic VAE Example](https://github.com/pytorch/examples/tree/master/vae)

7. [Deep Convolutional Generative Adversarial Network](https://www.tensorflow.org/tutorials/generative/dcgan)

8. [Generative adversarial networks: What GANs are and how they’ve evolved](https://venturebeat.com/2019/12/26/gan-generative-adversarial-network-explainer-ai-machine-learning/)

9. [Generative Adversarial Nets](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf)

10. [GANs by google](https://developers.google.com/machine-learning/gan)

11. [A Gentle Introduction to Generative Adversarial Networks (GANs)](https://machinelearningmastery.com/what-are-generative-adversarial-networks-gans/)

12. [A Beginner's Guide to Generative Adversarial Networks (GANs)](https://pathmind.com/wiki/generative-adversarial-network-gan)

13. [Understanding Generative Adversarial Networks (GANs)](https://towardsdatascience.com/understanding-generative-adversarial-networks-gans-cd6e4651a29)

14. [Deep Learning (PyTorch)](https://github.com/udacity/deep-learning-v2-pytorch)

15. [10 Lessons I Learned Training GANs for one Year](https://towardsdatascience.com/10-lessons-i-learned-training-generative-adversarial-networks-gans-for-a-year-c9071159628)